In [1]:
pip install openai


   ---------------------------------------- 266.9/266.9 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 75.6/75.6 kB ? eta 0:00:00
   ---------------------------------------- 77.9/77.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import time
import os

# Assuming the OpenAI and other necessary imports and setup from your initial code...

from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd
from tqdm import tqdm
import time

OPENAI_API_KEY=''

client = OpenAI(api_key=OPENAI_API_KEY)

def getChatGPT35Response(systemInstructions, prompt):
    # sleep to avoid api limits, should take ~15 minutes in total
    time.sleep(1)
    #return 'something'
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": f"{systemInstructions}"},
            {"role": "user", "content": f"{prompt}"}
        ]
    )

    return completion.choices[0].message.content

def getLLMResponse(systemInstructions, prompt):
    return getChatGPT35Response(systemInstructions, prompt)

def getBaseAnswers(dataDf):
    BASE_ANS_DIR = './results.json'

    saveDf = None#pd.DataFrame(columns=['question', 'reasoning', 'subproblems', 'label'])
    #saveDf = #saveDf.astype(str)
    
    for index, row in tqdm(dataDf.iterrows(), total=len(dataDf)):
        systemInstructions = "You are a helpful bot that can answer reasoning questions based off board game sitations"
        prompt = row['example'] + "\n The label is what (proved, disproved, unknown)?"
        response = getLLMResponse(systemInstructions, prompt)
        if saveDf is None:
            saveDf = pd.DataFrame(columns=['question', 'reasoning', 'subproblems', 'label'])
            saveDf = saveDf.astype(str)
        else:
            saveDf = pd.read_json(BASE_ANS_DIR, dtype=str)

        new_record = {
            'question': row['example'],
            'gold-reasoning': row['proof'],
            'subproblems': '',
            'chat-gpt-3.5-turbo-ans': response,
            'label': row['label'],
        }
        temp_df = pd.DataFrame([new_record])
        saveDf = pd.concat([saveDf, temp_df], ignore_index=True)
        saveDf.to_json(BASE_ANS_DIR, orient='records')






def createCOTPrompt(facts, rules, example, proof, goal, theory):
    """
    Construct a chain of thought prompt using the example and proof from the data.
    """
    # The prompt starts with a breakdown of the problem into facts and rules
    prompt = (
        f"Here are the facts: {facts}\n"
        f"And here are the rules: {rules}\n"
        f"Based on this, let's think through if {goal} is true.\n\n"
        f"{example}\n"
    )
    
    # The chain of thought is guided by the proof, ending with the goal
    cot = (
        f"Let's start with the chain of thought:\n{proof}\n\n"
        f"Now, given this chain of thought, let's answer the question: does {goal}?"
    )
    
    return prompt + cot

def getBaseAnswersWithCOT(dataDf):
    BASE_ANS_DIR = './results_chain_of_thought.json'
    saveDf = pd.DataFrame(columns=['question', 'gold-reasoning', 'label', 'chain_of_thought_response'])

    for index, row in tqdm(dataDf.iterrows(), total=dataDf.shape[0]):
        systemInstructions = "You are a helpful assistant capable of reasoning through complex problems."
        # Constructing the COT prompt with the example and proof from the data
        cot_prompt = createCOTPrompt(
            row['facts'], row['rules'], row['example'], row['proof'], row['goal'], row['theory']
        )
        response = getLLMResponse(systemInstructions, cot_prompt)

        # Create a new record for saving the response with chain of thought
        new_record = {
            'question': row['example'],
            'gold-reasoning': row['proof'],
            'label': row['label'],
            'chain_of_thought_response': response,
        }
        temp_df = pd.DataFrame([new_record])
        saveDf = pd.concat([saveDf, temp_df], ignore_index=True)

        # Sleep to avoid hitting API rate limits, if needed
        time.sleep(1)

    saveDf.to_json(BASE_ANS_DIR, orient='records', force_ascii=False)

# Example usage
if __name__ == "__main__":
    # Path to the dataset
    dataPath = "../data/test.json"
    
    # Load the dataset
    df = pd.read_json(dataPath, dtype=str)

    # Run the COT prompting process
    getBaseAnswersWithCOT(df)


100%|██████████| 1000/1000 [1:08:13<00:00,  4.09s/it]
